In [2]:
import openmm
from openmm import unit
from openmm import app
import numpy as np
from openff.toolkit.topology import Molecule

from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from sys import stdout
import mdtraj as md
#from openmmtools import integrators

In [3]:
proline = Molecule.from_pdb_and_smiles('Pro.pdb','C1C[C@H](NC1)C(=O)O')
#proline = Molecule.from_topology()

ff = app.ForceField()
#print(SMIRNOFFTemplateGenerator.INSTALLED_FORCEFIELDS)
smirnoff = SMIRNOFFTemplateGenerator(molecules = proline,forcefield='openff-2.0.0')

ff.registerTemplateGenerator(smirnoff.generator)
# #positions = proline.conformers

pdb = app.PDBFile('Pro.pdb')
# topology = pdb.topology
# positions = pdb.positions

system = ff.createSystem(
    topology=pdb.getTopology(), 
    removeCMMotion=True,
    nonbondedMethod=app.NoCutoff,
    constraints=app.HBonds, 
    rigidWater=True
    )

/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/abbie/miniconda3/envs/openmm7.7_openff_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
xml = openmm.XmlSerializer.serialize(system)
xml_file = open('proline_xmlsystem.txt','w')
xml_file.write(xml)
xml_file.close()

In [4]:
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 2.0 * unit.femtosecond
reportInterval = 2000
steps = 5E+8
#time = (steps*timestep).value_in_unit(unit.nanosecond)
# parameters = f'{temperature}\nCollision rate:{collision_rate}\nTimestep: {timestep}\nReport every {reportInterval} steps'
# with open('parameterfile.txt', 'w') as f:
#     f.write(parameters)
# parametersdict = {'Collision rate':collision_rate,'Temperature':temperature,'Timestep':timestep,'Report Interval':reportInterval}
# import pickle
# f_p = open(f'parameters_proline.pkl','wb')
# pickle.dump(parametersdict,f_p)
# f_p.close

<function BufferedWriter.close>

In [4]:
integrator = openmm.LangevinIntegrator(temperature,collision_rate,timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
platform = openmm.Platform.getPlatform(2)

pdb = app.PDBFile('Pro.pdb')
topology = pdb.getTopology()
positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

simulation = app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
#print(platform.getPropertyValue(simulation.context,property='Precision'))
#print(simulation.context.getPlatform())
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

simulation.reporters.append(app.StateDataReporter(stdout, reportInterval, step=True, potentialEnergy=True,temperature=True))
simulation.reporters.append(app.DCDReporter('proline_trajectory.dcd',reportInterval))
#h5_reporter = reporters.HDF5Reporter('output.h5',reportInterval)
#simulation.reporters.append(h5_reporter)

In [5]:
simulation.step(steps)
#h5_reporter.close()

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
2000,255.79827880859375,272.83681166263335
4000,276.3270263671875,412.6435994063051
6000,269.9858093261719,243.40833263658988
8000,277.60833740234375,310.0119317360376
10000,266.3017578125,372.8279683067858
12000,274.2093200683594,259.88796271081924
14000,269.88043212890625,252.9195858339357
16000,275.10357666015625,219.49823194314374
18000,251.0987548828125,379.7585002169119
20000,258.7463684082031,372.38144113456826
22000,276.9962158203125,335.34017916224104
24000,255.13458251953125,398.70269483064226
26000,256.9656982421875,315.1157916401338
28000,258.267333984375,279.152029238446
30000,284.4207763671875,163.86013280627844
32000,274.906494140625,443.47792283044544
34000,254.54806518554688,290.47390820606216
36000,245.36111450195312,306.48703398525026
38000,261.5446472167969,271.6405160343891
40000,276.3798828125,224.30430054487996
42000,253.19436645507812,236.56936116845122
44000,256.7413330078125,318.7028973100723
46000,278.366

In [ ]:
import mdtraj as md

#traj = md.load('pro_trajectory.dcd',top=md.Topology.from_openmm(topology))
traj = md.load('proline_trajectory.dcd',top='Pro.pdb')
import nglview
view = nglview.show_mdtraj(traj)
view